In [1]:
import pandas as pd 
import numpy as np

In [2]:
data_raw = pd.read_excel("data.xlsx")
weights = data_raw[["Weight"]]
scenarios = data_raw.drop(columns = ["Weight", "Unnamed: 36"])

In [3]:
from entropy_pooling import views

(A,b,C,d) = views.load();

     pcost       dcost       gap    pres   dres
 0: -1.9348e+08 -1.9348e+08  9e-01  2e+00  4e-19
 1: -1.9348e+08 -1.9348e+08  1e-02  1e-01  6e-20
 2: -1.9348e+08 -1.9348e+08  1e-04  9e-04  1e-19
 3: -1.9348e+08 -1.9348e+08  1e-06  9e-06  6e-20
 4: -1.9348e+08 -1.9348e+08  1e-08  9e-08  6e-20
Optimal solution found.


In [13]:
from entropy_pooling import entropy_minimizer
c = 0.5
posterior = entropy_minimizer.full_confidence_posterior(A,b,C,d)
weighted_posterior = entropy_minimizer.confidence_weighted_posterior(weights, posterior, c)

TypeError: full_confidence_posterior() missing 1 required positional argument: 'd'

In [40]:
from entropy_pooling import markoviz_optimizer
optimal_x = markoviz_optimizer.optimizer(scenarios, weighted_posterior, mu_0 = 0.5, disp = False, vizualization = True).x

TypeError: optimizer() got an unexpected keyword argument 'vizualization'